In [1]:
# Import des librairies nécessaires 
import pandas as pd
import numpy as np
import time
import re
%matplotlib inline
from tqdm import tqdm
from urllib.request import urlopen,Request
from bs4 import BeautifulSoup
from googlesearch import search


In [2]:
# Installation et import des outils NordVpn permettant de tourner sur plusieurs serveurs VPN
# Choix de la localisation des serveurs : France et Belgique
!pip install nordvpn-switcher
from nordvpn_switcher import initialize_VPN,rotate_VPN,terminate_VPN
settings=initialize_VPN(area_input=['France','Belgium'])

You're using Windows.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN service check: ✓
Opening NordVPN app and disconnecting if necessary...
NordVPN app launched: ✓
#####################################

You've entered a list of connection options. Checking list...


Done!



In [3]:
#Lecture fichier et initialisation colonne top (indicateur de traitement)
df = pd.read_csv(r"vgsales.csv")
df['top']=''''

In [6]:
# Fonction de lecture par balise classe via Beautiful soup
def lec_class_soup(classe,soup):
    result='not found'
    x= soup.select(classe)
    for n in x:
        result=n.text
    return(result)

In [7]:
#Fonction de lecture du fichier initial, scraping de métacritic
# et ajout des colonnes :
#metascorewrap : metascore (score + nb critiques) issus de metacritic, basé sur une synthèse des critiques
#sur les différents sites de jeux vidéo
#userscorewrap : note des utilisateurs et nombre de votes associés
#gamename : nom du jeu pour vérification de concordance
#developper : studio de développement

def scrape_and_fill(df):
    starting_index = df[df['top'] == ""].index[0]
    roll=1 # roll=1, on change de serveur VPN à chaque appel de la fonction
    #Lecture du dataset en débutant à la premiere ligne non traitée
    for i, row in tqdm(df.loc[starting_index:].iterrows(), total = len(df) - starting_index):
            #Changement de serveur VPN toutes les 20 lignes ou en cas d'erreur html
            if (((i%20)==0) or (roll==1)):
                rotate_VPN(settings,google_check=1)
                fichier='vgsales2_'+str(i)+'.csv' #fichier intermédiaire de sauvegarde
                df.to_csv(fichier, index = False)
                roll=0
            time.sleep(np.random.randint(2, 10)) #temprisation aléatoire pour simuler utilisation humaine
            try: # lecture site metacritic
                for url in search('metacritic.com/game '+row['Name'], tld='com', start=0, stop=1, pause=2.0):
                  if ('metacritic' in url):  # Vérification du site
                    req = Request(url,headers={'User-Agent': 'Chrome/94.0.4606.71'})
                    page_criti = urlopen(req).read()
                    soup = BeautifulSoup(page_criti, 'html.parser')
                    #initialisation données
                    df.loc[i, 'metascorewrap'] = lec_class_soup('.metascore_wrap',soup)
                    df.loc[i, 'userscorewrap'] = lec_class_soup('.userscore_wrap',soup)
                    df.loc[i, 'gamename'] = lec_class_soup('h1',soup)
                    df.loc[i, 'top'] = 1                                                           
                    developper='not found'
                    x=soup.findAll(name = 'a', attrs = {'class':'button'})
                    for n in x:
                        developper=n.text
                    df.loc[i, 'developper'] = developper
                  else: #Données non trouvées
                     df.loc[i, 'metascorewrap'] = df.loc[i, 'userscorewrap'] =  'not found'
                     df.loc[i, 'gamename'] = df.loc[i, 'developper'] = 'not found'                                                           
                     df.loc[i, 'top'] = 1
                    
            except(RuntimeError, TypeError, NameError):
                fichier='vgsales2_'+str(i)+'.csv' #fichier intermédiaire de sauvegarde
                df.to_csv(fichier, index = False)
                scrape_and_fill(df) # Si erreur à l'exécution ,rappel récursif de la fonction au dernier point traité
            except:
                fichier='vgsales2_'+str(i)+'.csv'
                df.to_csv(fichier, index = False) #fichier intermédiaire de sauvegarde
                scrape_and_fill(df) # Si erreur à l'exécution ,rappel récursif de la fonction au dernier point traité
    df.to_csv("vgsales2_new.csv", index = False)
    return df


In [ ]:
scrape_and_fill(df)